In [1]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.system("mkdir -p data/output/fig")

fontsize = 22
labelsize = 18

In [2]:
df = pd.read_csv("data/output/result.csv", header=None)
df.columns = ['algo', 'bucket update time', 'core', 'dataset', 'degree call time', 'execution time', 'neighborhood call time', 'num bucket update', 'num degree computation', 'num neighborhood computation', 'param_s']
print(df.shape)
df

(39, 11)


,algo,bucket update time,core,dataset,degree call time,execution time,neighborhood call time,num bucket update,num degree computation,num neighborhood computation,param_s
0,naive_nbr,0.000030,"{'7': 1, '1': 3, '6': 3, '9': 3, '8': 3, '5': ...",syn,0.000000,0.006837,0.000262,15.0,0.0,15.0,0.0
1,improved2_nbr,0.000014,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.011451,0.000268,11.0,0.0,25.0,9.0
2,improved2_nbr,0.000024,"{'8': 3, '1': 3, '6': 3, '9': 3, '10': 3, '5':...",syn,0.000000,0.010486,0.000446,10.0,0.0,25.0,1.0
3,improved2_nbr,0.000035,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.012290,0.000592,11.0,0.0,25.0,3.0
4,improved2_nbr,0.000038,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.012136,0.000588,11.0,0.0,25.0,7.0
5,naive_degree,0.000024,"{'6': 1, '7': 1, '5': 1, '8': 1, '9': 2, '1': ...",syn,0.000085,0.005307,0.000000,15.0,15.0,0.0,0.0
6,improved2_nbr,0.000037,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.011744,0.000591,11.0,0.0,25.0,5.0
7,improved_nbr,0.000015,"{'7': 1, '1': 3, '6': 3, '9': 3, '8': 3, '5': ...",syn,0.000000,0.004046,0.000147,0.0,0.0,15.0,0.0
8,improved2_nbr,0.000018,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.006875,0.000276,11.0,0.0,25.0,10.0
9,improved2_nbr,0.000042,"{'7': 1, '8': 3, '1': 3, '6': 3, '9': 3, '10':...",syn,0.000000,0.015726,0.000715,11.0,0.0,25.0,6.0


### sample plotting

In [22]:
save = True


sns.set(rc={'figure.figsize':(5,4)})
sns.set_style("whitegrid", {'axes.grid' : True})



good_name = {
    'bucket update time' : 'Bucket time (s)', 
    'execution time' : "Time (s)", 
    'neighborhood call time' : "NB time (s)", 
    'num bucket update' : "#bucket", 
    'num neighborhood computation' : "#nb"
}

good_name_algo = {
    'naive_nbr' : "NBR", 
    'improved_nbr' : "NBR(1)", 
    'improved2_nbr' : "NBR(2)",
    'naive_degree' : "DEG"
}




group_list = ['dataset']
for key, item in df.groupby(group_list, as_index=False):
    # key contains dataset
    item['param_s'] = item['param_s'].astype(int)
    item = item.replace({"algo": good_name_algo})
    for y_axis in ['bucket update time', 'execution time', 'neighborhood call time', 'num bucket update', 'num neighborhood computation']:

        sns.barplot(x='algo', y=y_axis, data=item, palette='colorblind', order=['NBR', 'NBR(1)', 'NBR(2)'])
        plt.xlabel('Algorithm', fontsize=fontsize)
        plt.ylabel(good_name[y_axis], fontsize=fontsize)
        # plt.yscale("log")
        plt.xticks(fontsize=labelsize)
        plt.yticks(fontsize=labelsize)
        plt.title(key)
        plt.tight_layout()
        filename = (key + " " + y_axis).replace(" ", "_")
        if(save):
            plt.savefig("data/output/fig/" + filename + ".pdf")
        else:
            print(filename)
            plt.show()
        plt.clf()

    # Commnet out following to see full results
    if(not save):
        break


<Figure size 432x288 with 0 Axes>

### Effect of parameter "s" in improved2_nbr

In [33]:
save = True

group_list = ['dataset']
for key, item in df[df['algo'] == 'improved2_nbr'].groupby(group_list, as_index=False):
    
    # key contains dataset
    item['param_s'] = item['param_s'].astype(int)

    for y_axis in ['bucket update time', 'execution time', 'neighborhood call time', 'num bucket update', 'num neighborhood computation']:

        sns.barplot(x='param_s', y=y_axis, data=item, palette='colorblind')
        plt.xlabel(r'$s$', fontsize=fontsize)
        plt.ylabel(good_name[y_axis], fontsize=fontsize)
        # plt.yscale("log")
        plt.xticks(fontsize=labelsize)
        plt.yticks(fontsize=labelsize)

        plt.title(key)
        # plt.legend(loc='best', fontsize=labelsize-4)
        plt.tight_layout()
        filename = (key + " param_s " + y_axis).replace(" ", "_")
        if(save):
            plt.savefig("data/output/fig/" + filename + ".pdf")
        else:
            print(filename)
            plt.show()
        plt.clf()

    # Commnet out following to see full results
    if(not save):
        break


<Figure size 432x288 with 0 Axes>

In [28]:
df.columns

Index(['algo', 'bucket update time', 'core', 'dataset', 'degree call time',
       'execution time', 'neighborhood call time', 'num bucket update',
       'num degree computation', 'num neighborhood computation', 'param_s'],
      dtype='object')